<a href="https://colab.research.google.com/github/jtbigdata/VentaDepartamentos_SgoChile/blob/main/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#!pip3 install dash
#!pip3 install dash-bootstrap-components
#!pip3 install dash-bootstrap-templates
#!pip3 install dash-ag-grid

In [10]:
from dash import Dash, dcc, html, Input, Output, State, callback, Patch, clientside_callback
import plotly.express as px
import plotly.io as pio
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import ThemeChangerAIO, template_from_url
import dash_ag_grid as dag
import json
import pandas as pd
import datetime

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/VentaDpto_StgoChile/Departamentos_Chile/Dashboard/DeptoChile_Dashboard7.csv')

In [26]:
# stylesheet with the .dbc class to style  dcc, DataTable and AG Grid components with a Bootstrap theme
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
#================================DEFINO COLUMNAS==================================================================================
df['Fecha_de_publicación'] = pd.to_datetime(df['Fecha_de_publicación'])
# Luego, convierte las fechas al formato deseado '%Y-%m'
df['Fecha_de_publicación'] = df['Fecha_de_publicación'].dt.strftime('%Y-%m')
FechasP=sorted(df.Fecha_de_publicación.unique())

FechasP = [datetime.datetime.strptime(fecha, '%Y-%m').strftime('%Y-%m') for fecha in FechasP]


Vcont = ['Precio_CLP', 'Gastos_comunes', 'Superficie_total', 'Superficie_útil','Precio_x_m2tot', 'Precio_x_m2util']
Vtemp = ['Fecha_de_publicación', 'Decada_de_construccion']
Vdisc = ['Dormitorios','Baños', 'Estacionamiento', 'Cocina', 'Terraza', 'Bodega', 'Piscina']
Vdiscb=['Zona','Agente_inmobiliario']
Vcomu=df.Comuna.unique()

In [29]:
#=========================================INICIO APP========================================================================
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME, dbc_css],
           #uppress_callback_exceptions=True
           )
#---------switch modo nocturno+diurno
color_mode_switch =  html.Span(
    [
        dbc.Label(className="fa fa-moon", html_for="switch"),
        dbc.Switch( id="switch", value=True, className="d-inline-block ms-1", persistence=True),
        dbc.Label(className="fa fa-sun", html_for="switch"),
    ]
)

#------The ThemeChangerAIO loads all 52  Bootstrap themed figure templates to plotly.io
#--------controles para cambiar el tema de las figuras generadas por Plotly.
theme_controls = html.Div(
    [ThemeChangerAIO(aio_id="theme"), color_mode_switch],
    className="hstack gap-3 mt-2"
)

header = html.H1(
    "Venta de Departamentos en Santiago de Chile.", className="bg-primary text-white p-2 mb-2 text-center"
)

#==========================Controladores tab1+tab2=================================

# Define el estilo para el tamaño de fuente y del dropdown
texto_style = {'fontSize': '0.6rem', 'maxHeight': '30px'}  # Modifica el valor de 'maxHeight' según el tamaño deseado
dropdown_width = {'width': '130px'}  # Define el ancho deseado para los dropdowns
dropdown_width2 = {'width': '130px'}  # Define el ancho deseado para los dropdowns

subtitle1 = html.H4("Tab1 y Tab2", className="bg-primary text-white p-2 mb-0.1 text-center", style=texto_style)

dropdown_a1 = html.Div([dbc.Label("Y-axis", style=texto_style),
    dcc.Dropdown(Vcont,'Precio_x_m2util',id="id1",clearable=False, style={**texto_style, **dropdown_width}),],
    className="mb-0.1 mr-2",
)

dropdown_a2 = html.Div([dbc.Label("X-axis", style=texto_style),
    dcc.Dropdown(Vtemp,Vtemp[0],id="id2",clearable=False, style={**texto_style, **dropdown_width}),
    ],className="mb-0.1",
)

controls1 = dbc.Card(
    [subtitle1, html.Div([dropdown_a1, dropdown_a2], className="d-flex flex-row justify-content-around align-items-center")],
    body=True,style={'height': '125px'}
)



#==========================Controladores tab3+tab4=================================

subtitle2 = html.H4("Tab3 y Tab4", className="bg-primary text-white p-2 mb-0.1 text-center", style=texto_style)

dropdown_a3 = html.Div([dbc.Label("Y-axis", style=texto_style),
    dcc.Dropdown(Vdisc,Vdisc[0],id="id3",clearable=False, style={**texto_style, **dropdown_width}),],
    className="mb-0.1 mr-2",
)

dropdown_a4 = html.Div([dbc.Label("X-axis", style=texto_style),
    dcc.Dropdown(Vdisc,Vdisc[1],id="id4",clearable=False, style={**texto_style, **dropdown_width}),
    ],className="mb-0.1",
)

controls2 = dbc.Card(
    [subtitle2, html.Div([dropdown_a3, dropdown_a4], className="d-flex flex-row justify-content-around align-items-center")],
    body=True,style={'height': '125px'}
)


#==========================Controladores tab5=================================
subtitle3= html.H4("Tab5", className="bg-primary text-white p-2 mb-0.01 text-center",style=texto_style
)

dropdown_a5 = html.Div(
    [#dbc.Label("Comuna",style=texto_style),
     dcc.Dropdown(Vcomu,Vcomu[0],id="id5",clearable=False,style={**texto_style, **dropdown_width}),],
    className="mb-0.1",
)

controls3 = dbc.Card([subtitle3,dropdown_a5],body=True,style={'height': '100px'})


#==========================Controladores tab6=================================
subtitle4= html.H4("Tab6", className="bg-primary text-white p-2 mb-2 text-center",style=texto_style
)

a=df.Baños.unique()
b=df.Dormitorios.unique()
c=df.Precio_CLP.unique()
a.sort()
b.sort()
c.sort()

dropdown_a6 = html.Div(
    [dbc.Label("Baños",style=texto_style),
     dcc.Dropdown(a,[a[0],a[1]],id="id6",multi=True,clearable=False,style={**texto_style, **dropdown_width2}),],
    className="mb-0.1",
)

dropdown_a7 = html.Div(
    [dbc.Label("Dormitorios",style=texto_style),
     dcc.Dropdown(b,[b[0],b[1]],id="id7",multi=True,clearable=False,style={**texto_style, **dropdown_width2}),],
    className="mb-0.1",
)


# Crear los controles dentro del Card
controls4 = dbc.Card(
    [subtitle4,
    #html.Div([dropdown_a6, dropdown_a7], className="d-flex flex-row justify-content-around align-items-center"),
    html.Div([dropdown_a6, dropdown_a7], className="d-flex flex-row justify-content-around align-items-center", style={"margin-bottom": "25px"}),
    #sliders
    ],
    body=True,
    style={'height': '125px'}
)

#====================TabsTodas========================================================

tab1 = dbc.Tab([dcc.Graph(id="bar-graph" , figure=px.bar(template="bootstrap")),], label="Tab1",style={'height': '475px'})
tab2 = dbc.Tab([dcc.Graph(id="line-chart", figure=px.line(template="bootstrap"))], label="Tab2", style={'height': '475px'})
tab3 = dbc.Tab([dcc.Graph(id="density_heatmap1" , figure=px.density_heatmap(template="bootstrap")),], label="Tab3", style={'height': '475px'})  # Añade el estilo height: 100%
tab4 = dbc.Tab([dcc.Graph(id="density_heatmap2", figure=px.density_heatmap(template="bootstrap"))], label="Tab4", style={'height': '475px'})  # Añade el estilo height: 100%
tab5 = dbc.Tab([dcc.Graph(id="scatter", figure=px.scatter(template="bootstrap"))], label="Tab5", style={'height': '475px'})  # Añade el estilo height: 100%
tab6 = dbc.Tab([dcc.Graph(id="Map", figure=px.choropleth_mapbox(template="bootstrap"))], label="Tab6", style={'height': '475px'})  # Añade el estilo height: 100%

tabs = dbc.Card(dbc.Tabs([tab1, tab2, tab3, tab4, tab5, tab6]), style={'height': '475px'})  # Añade el estilo height: 100%
#==========================DEFINO el Layout=======================================

app.layout = dbc.Container(
    [header,
        dbc.Row([
            dbc.Col([controls1,controls2,controls3,controls4,theme_controls],  width=4),
            dbc.Col([tabs], width=8),
        ])
    ],
    fluid=True, className="dbc dbc-ag-grid h-100",
)

In [30]:
#-------------------------CALLBACK TAB1--------------------------------------
@callback(
    Output("bar-graph", "figure" ),
    Input("id1", "value"),
    State(ThemeChangerAIO.ids.radio("theme"), "value"),
    State("switch", "value"),
)

def update_graph(id1,theme, color_mode_switch_on):
    global df
    if id1 == [] or None:
        return {}, {}, {}
    theme_name = template_from_url(theme)
    template_name = theme_name if color_mode_switch_on else theme_name + "_dark"

    df_a = df.groupby(['Comuna'])[id1].mean().reset_index(name='mean')


    fig = px.bar(df_a, x='Comuna',y='mean', color='Comuna', #marginal="rug", # can be `box`, `violin`
                         hover_data=df_a.columns)

    return fig
#--------------------------------CALLBACKTab2---------------------------------------------------------------------------------

@callback(
    Output("line-chart", "figure" ),
    Input("id1", "value"),
    Input("id2", "value"),
    State(ThemeChangerAIO.ids.radio("theme"), "value"),
    State("switch", "value"),
)

def update_graph(id1, id2,theme, color_mode_switch_on):
    global df
    if id1 == [] or id2 is None:
        return {}, {}, {}
    theme_name = template_from_url(theme)
    template_name = theme_name if color_mode_switch_on else theme_name + "_dark"

    if id2=='Fecha_de_publicación':
        df_a = df.loc[:, ['Fecha_de_publicación', 'Comuna', id1]]
        df_a.loc[:, 'Fecha_de_publicación'] = pd.to_datetime(df_a['Fecha_de_publicación']).dt.strftime('%Y-%m')

        df_a2 = df_a.groupby(['Fecha_de_publicación', 'Comuna'])[id1].mean().reset_index(name='mean')


        fig = px.line(df_a2, x='Fecha_de_publicación', y='mean', color='Comuna', markers=True)

        return fig
    elif id2 =='Decada_de_construccion':
        df_b = df.loc[:, ['Decada_de_construccion', 'Comuna', id1]]
        df_b=df_b[df_b['Decada_de_construccion']>int(2000)]
        df_b=df_b[df_b['Decada_de_construccion']<int(2025)]

        df_b2 = df_b.groupby(['Decada_de_construccion', 'Comuna'])[id1].mean().reset_index(name='mean')


        fig = px.line(df_b2, x='Decada_de_construccion', y='mean', color='Comuna', markers=True)


        return fig
#-------------------------CALLBACK TAB3------------------------------
@callback(
    Output("density_heatmap1", "figure" ),
    Input("id3", "value"),
    Input("id4", "value"),
    State(ThemeChangerAIO.ids.radio("theme"), "value"),
    State("switch", "value"),
)


def update_graph(id3,id4,theme, color_mode_switch_on):
    if id3 == [] or id4 is None:
        return {}, {}, {}
    theme_name = template_from_url(theme)
    template_name = theme_name if color_mode_switch_on else theme_name + "_dark"

    fig = px.density_heatmap(df,x=id3, y=id4 ,z='Precio_CLP'
                             , histfunc="count",color_continuous_scale='viridis')

    return fig


#-------------------------CALLBACK TAB4------------------------------
@callback(
    Output("density_heatmap2", "figure" ),
    Input("id3", "value"),Input("id4", "value"),
    State(ThemeChangerAIO.ids.radio("theme"), "value"),
    State("switch", "value"),
)


def update_graph(id3,id4,theme, color_mode_switch_on):
    global df
    if id3 == [] or id4 is None:
        return {}, {}, {}
    theme_name = template_from_url(theme)
    template_name = theme_name if color_mode_switch_on else theme_name + "_dark"

    # Calcular el valor medio de la columna
    valor_medio = df['Precio_x_m2util'].mean()

    # Rellenar los valores NaN con el valor medio
    df['Precio_x_m2util'] = df['Precio_x_m2util'].fillna(valor_medio)
    fig = px.density_heatmap(df, x=id3, y=id4, z='Precio_x_m2util', histfunc="avg",color_continuous_scale='viridis')

    return fig



#-------------------------CALLBACK TAB5------------------------------
@callback(
    Output("scatter", "figure" ),
    Input("id5", "value"),
    State(ThemeChangerAIO.ids.radio("theme"), "value"),
    State("switch", "value"),
)

def update_graph(id5,theme, color_mode_switch_on):
    global df
    if id5 == [] or None:
        return {}, {}, {}
    theme_name = template_from_url(theme)
    template_name = theme_name if color_mode_switch_on else theme_name + "_dark"

    dfa=df[df['Comuna']==id5]

    fig = px.scatter(dfa, x='Superficie_total', y='Superficie_útil',
                 color='Precio_CLP', size='Precio_CLP',
                 labels={'Superficie_total': 'Superficie total', 'Gastos_comunes': 'Gastos comunes', 'Precio_x_m2util': 'Precio x m2 util'},
                 color_continuous_scale='viridis',
                 range_color=[dfa['Precio_CLP'].min(), dfa['Precio_CLP'].max()],
                 color_continuous_midpoint=dfa['Precio_CLP'].mean()  # Puedes ajustar el punto medio de la escala de colores si lo deseas
                )
    return fig




#-------------------------CALLBACK TAB6------------------------------
@callback(
    Output("Map", "figure" ),
    Input("id6", "value"),Input("id7", "value"),#Input("id8", "value"),
    State(ThemeChangerAIO.ids.radio("theme"), "value"),
    State("switch", "value"),
)



def update_graph(id6,id7,theme, color_mode_switch_on):
    global df
    if id6 == [] or None:
        return {}, {}, {}
    if id7 == [] or None:
        return {}, {}, {}
    #if id8 == [] or None:
    #    return {}, {}, {}


    theme_name = template_from_url(theme)
    template_name = theme_name if color_mode_switch_on else theme_name + "_dark"
    #global df

    df=df[df['Baños'].isin(id6)]
    df=df[df['Dormitorios'].isin(id7)]

    # Calcular la media por comuna para cada una de las cuatro columnas originales
    df_media = df.groupby('Comuna').agg({'Precio_CLP':'mean','Gastos_comunes':'mean',
                                                 'Precio_x_m2util': 'mean','Precio_x_m2tot': 'mean',
                                                'Superficie_útil': 'mean','Superficie_total': 'mean'}).reset_index()
    df_media = df_media.round(2)

# Mostrar las primeras filas del DataFrame con la media por comuna para verificar
    ruta_archivo = 'all.geojson'
    # Cargar los datos GeoJSON que contienen los límites de las comunas de Santiago
    with open(ruta_archivo, 'r') as f:
        geojson_data = json.load(f)

    # Crear el mapa de calor utilizando Plotly Express

    print('===AKA=',df_media.head(8))
    fig = px.choropleth_mapbox(df_media,
                                geojson=geojson_data,
                                locations='Comuna',
                                featureidkey="properties.NOM_COM",
                                color='Precio_CLP',  # Columna que contiene los precios
                                color_continuous_scale="Viridis",  # Escala de colores
                                mapbox_style="carto-positron",  # Estilo del mapa
                                center={"lat": -33.4372, "lon": -70.6506},  # Centro del mapa
                                zoom=10,  # Zoom inicial
                                opacity=0.5,  # Opacidad de los polígonos
                                labels={str(id6): str(id6)},  # Etiqueta del colorbar
                                title='Mapa de Calor de Precios en Santiago de Chile',  # Título del gráfico
                                hover_data={'Comuna': True,
                                            'Precio_CLP': True,
                                            'Superficie_útil':True,
                                            'Gastos_comunes':True,
                                            'Superficie_total':True,
                                            'Precio_x_m2tot':True,
                                            'Precio_x_m2util':True,
                                            }  # Datos adicionales al pasar el cursor
                            )
    fig.update_layout(
    title_text='Precios en las Comunas de Santiago de Chile',  # Título del mapa
    title_x=0.5,  # Posición horizontal del título (0.5 es el centro)
    title_y=0.95,  # Posición vertical del título (0.95 es cercano al tope del gráfico)
    title_font=dict(size=20,  # Tamaño de la fuente del título
                    color='black')
)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
     #Leyenda--> Barra de calor de cada sub-grafico
    fig.update_layout(coloraxis_colorbar=dict(title='Avg Precio_CLP',  # Agregar título a la barra de color
                                          title_font=dict(size=16,  # Tamaño de la fuente del título
                                                          color='black'),  # Color del título
                                          xanchor='center',  # Centrar horizontalmente el título
                                          x=1.1 # Posición horizontal del título (0.5 es el centro)
                                         )
                 )
    return fig




#========================================================================

# updates the Bootstrap global light/dark color mode
clientside_callback(
    """
    switchOn => {
       document.documentElement.setAttribute('data-bs-theme', switchOn ? 'light' : 'dark');
       return window.dash_clientside.no_update
    }
    """,
    Output("switch", "id"),
    Input("switch", "value"),
)


# This callback makes updating figures with the new theme much faster
@callback(
    Output("line-chart", "figure", allow_duplicate=True ),
    Output("bar-graph", "figure", allow_duplicate=True ),
    Output("density_heatmap1", "figure", allow_duplicate=True ),
    Output("density_heatmap2", "figure", allow_duplicate=True ),
    Output("scatter", "figure", allow_duplicate=True),
    Output("Map", "figure", allow_duplicate=True),
    Input(ThemeChangerAIO.ids.radio("theme"), "value"),
    Input("switch", "value"),
    prevent_initial_call=True
)

def update_template(theme, color_mode_switch_on):
    theme_name = template_from_url(theme)
    template_name = theme_name if color_mode_switch_on else theme_name + "_dark"

    patched_figure = Patch()
    # When using Patch() to update the figure template, you must use the figure template dict
    # from plotly.io  and not just the template name
    patched_figure["layout"]["template"] = pio.templates[template_name]
    return patched_figure, patched_figure, patched_figure, patched_figure, patched_figure,patched_figure


if __name__ == "__main__":
    app.run_server(debug=False,inline=True)

<IPython.core.display.Javascript object>